In [ ]:
from langchain_mistralai import ChatMistralAI
import getpass
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate
import json

load_dotenv()

In [10]:
input_text = """
ark soy sauce
really tasty fried rice
marinade it adds a punch to remember
WheatAcidity Regulator (INS 260)Preservative INS 202
allergy advice contains wheat soy
may contain nuts sesame seeds mustard milk
keep refrigerated after opening
for sain india nepal bhutan bangladesh only
store in a cool dry and hygienic place
bath no
recipes at wow ching secret com
or
P130408509
in of all taxes
25(R5.0.289
09/05.24(01:47)
a of mfg
usury
0805125
villas
also ghana compound or oshawa a bridge
Lic.No.10013022001865
fora
plaint contact manager consumer
Manutactured byFFortune Foods td.Gut No 256/2
net were
Village Awankhed,Tal-Dindori,Nasik-422202
90g
Lic.No.10013022001948
plan foods div of kaye food industries pvt ltd
Lic.No.10012064000055
890159
Lic.No.10018062000413
things secretive
of capital
818
1102155
trademark
india"""

In [ ]:
from gpt.product_chain import get_gpt_formatted_text

get_gpt_formatted_text(input_text)

In [ ]:
from sqlalchemy import inspect, create_engine
import os
from dotenv import load_dotenv

from db.models import Base

load_dotenv()
POSTGRES_URL = os.getenv("POSTGRESQL_DB_URL")
print(POSTGRES_URL)
engine = create_engine(POSTGRES_URL)

# Base.metadata.create_all(bind=engine)

inspector = inspect(engine)
print(inspector.get_table_names())

In [ ]:
from db.models import Product
from db.database import get_sql_db

new_product = Product(name="testing product",
                      price = 20.00,
                      total_no = 5)
session = next(get_sql_db())

# session.add(new_product)
session.commit()

In [ ]:
products = session.query(Product).all()
for product in products:
    print(f"ID: {product.item_id}, Name: {product.name} Price: {product.price}")

In [29]:
import os
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

In [30]:
client = QdrantClient(url=os.getenv('VECTOR_DB_URL'))

In [31]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",cache_folder="./vector/embedding_models")

/home/blank/miniconda3/envs/grid/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [32]:
client.get_collection(
    collection_name="products",
    # vectors_config=VectorParams(size=384, distance=Distance.COSINE)
)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=2, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quant

In [12]:
from langchain_core.documents import Document

doc_1 = Document(
    page_content="Hello this is content that is getting put in",
    metadata={"test_id" : 1}
)
doc_2 = Document(
    page_content="bye this is content that is getting put in",
    metadata={"test_id" : 2}
)

docs = [doc_1, doc_2]

In [33]:
from langchain_qdrant import QdrantVectorStore

vector_store = QdrantVectorStore(
    client=client,
    embedding=embeddings,
    collection_name="products"
)

In [14]:
vector_store.add_documents(docs)

['0fa13bcafb1149f6b4cafa7493d972ae', '4e15eebad84e4a0eb9eb165db784b092']

In [34]:
res = vector_store.similarity_search("hi", k = 1)
print(res[0])

page_content='bye this is content that is getting put in' metadata={'test_id': 2, '_id': '4e15eeba-d84e-4a0e-b9eb-165db784b092', '_collection_name': 'products'}


In [35]:
res[0].metadata

{'test_id': 2,
 '_id': '4e15eeba-d84e-4a0e-b9eb-165db784b092',
 '_collection_name': 'products'}